# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [275]:
import urllib.request
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [276]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200 #max iteration so we avoid possible infinite loop
    e  = 0.001
    alpha = 1.

    w = np.random.randn(X.shape[1]); #random array with size of m
    for k in np.arange(max_iter): #loops through array (from 0 to 199)
        h = np.dot(X,w) #multiply X (2-Dimensional array) with w (1-Dimensional array) (computes prediction)
        l,lg = loss(h, y) #compute hinge loss and its gradient (h is prediction and y is true label)
        print ('loss: {}'.format(np.mean(l))) #print mean of the hinge loss so we can see progress of learning
        r,rg = reg(w, lbda) #computes the L_2-regularizer and the gradient of the regularizer function at point w
        g = np.dot(X.T,lg) + rg #calculate new gradient from loss and regularizer gradient
        if (k > 0): #check if we are in the first iteration of the loop
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old)) 
        w = w - alpha * g #adjust w by alpha * gradient
        if (np.linalg.norm(alpha * g) < e): #check if normalization of alpha * g smaller than 0.001
            break #model is shaped enough
        g_old = g #remember gradient from last iteration
    return w #return linear model

print(learn_reg_ERM(X_train,y_train,5))

loss: 0.9998767656955323
loss: 0.6021972441121234
loss: 0.5508417655792076
loss: 0.7613976083001255
loss: 0.5280607338838126
loss: 0.5332242136094736
loss: 0.8094581592380659
loss: 0.5327054265524052
loss: 0.5343537230207829
loss: 0.9199999999999994
loss: 0.5361962590045364
loss: 0.5393696712791997
loss: 0.5457701036153914
loss: 0.5505201194016545
loss: 0.55387803322199
loss: 0.5516037402767451
loss: 0.5725
loss: 0.5468603280447747
loss: 0.5487913396065814
loss: 0.5604112785984577
loss: 0.5472375848930031
loss: 0.550208597813352
loss: 0.5498906874502577
loss: 0.5511001142848109
loss: 0.5498518034737143
loss: 0.5503527619608404
loss: 0.5501934358887723
loss: 0.5494675278578774
loss: 0.5502176359418278
loss: 0.5497995174578564
loss: 0.5498013641828752
loss: 0.5498157442924378
loss: 0.5499108661757942
loss: 0.5497136068276693
loss: 0.5497439599264456
loss: 0.549748277073886
loss: 0.5496959182865101
[ 2.60796540e-01 -1.74030466e-18  1.57978804e-02  1.81179980e-01
  1.57425361e-01  1.786343

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [277]:
def loss(h, y):
    ##################
    #INSERT CODE HERE#
    ##################
    l = [] #list to collect values for the hinge loss vector
    g = [] #list to collect values for the gradient vector
    for i in range(0,h.size): #loop through function values of the linear model
        new = max(0,1 - y[i] * h[i]) #hinge loss of this iteration
        l.append(new) #add hinge loss to vector
        if new > 0: #check if hinge loss is bigger than 0
            g.append(-y[i]) 
        else:
            g.append(0)
    l = np.array(l) #cast to numpy array
    g = np.array(g) #cast to numoy array
    return l, g #return hinge loss and gradient vector

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [278]:
def reg(w, lbda):
    ##################
    #INSERT CODE HERE#
    ##################
    tmp = np.dot(w.T,w) #calculate temporary array
    r = np.dot(lbda / 2, tmp) #calculate L_2-regularizer
    g = np.dot(lbda, w) #calculate gradient of the regularizer function at point w
    return r, g #return regularizer and gradient

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [279]:
def predict(w, X):
    ##################
    #INSERT CODE HERE#
    ##################
    preds = np.dot(X,w) #multiply matrix with model
    for i in range(0,preds.size): #loop through prediction array
        if preds[i] < 0.5: #check to which label the value is closer
            preds[i] = 0 #prediction is closer to 0 than to 1
            continue
        preds[i] = 1 #prediction is closer to 1 than to 0
    return preds #return the prediction

print(predict(learn_reg_ERM(X_train,y_train,100), X_test)) #print prediction
print(y_test) #print actual label

loss: 2.0842673369284492
loss: 2.3903627702383408
loss: 0.9116958237267554
loss: 0.6143018979159074
loss: 0.7119685050629853
loss: 0.6894263750417838
loss: 0.6993824289316135
loss: 0.6968955001377835
loss: 0.6980683644873336
loss: 0.7026249999999999
loss: 0.6923036309136423
loss: 0.6960467602462445
loss: 0.695384324279192
loss: 0.6961930429016497
[1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0.
 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [280]:
##################
#INSERT CODE HERE#
##################
def accuracy(y_pred,y_true):
    misses = 0 #variable to count misses
    for i in range(0,y_pred.size): #loop through prediction array
        if y_pred[i] != y_true[i]: #check if the prediction is wrong
            misses += 1 #prediction is wrong we count one miss
    return (y_pred.size - misses) / y_pred.size #calculate accuracy

linear_model = learn_reg_ERM(X_train,y_train,2) #train model
lm_train_pred = predict(linear_model, X_train) #make prediction on training data
lm_test_pred = predict(linear_model, X_test) #make prediction on test data

#display accuracy
print("The accuracy with the linear model on the training data is:", accuracy(lm_train_pred,y_train))
print("The accuracy with the linear model on the test data is:", accuracy(lm_test_pred,y_test))

loss: 2.031642824882133
loss: 0.525
loss: 0.525
loss: 1.0000000000000036
loss: 0.525
loss: 0.525
loss: 1.0000000000000013
loss: 0.525
loss: 0.525
loss: 1.0
loss: 0.5252236888679076
loss: 0.52601683573058
loss: 0.9187500000000007
loss: 0.525
loss: 0.52523966621622
loss: 0.5675265750937862
loss: 0.5353664662864208
loss: 0.5359417983884318
loss: 0.8000000000000004
loss: 0.5282450762177258
loss: 0.528401138478495
loss: 0.5312770476954415
loss: 0.5308966043217739
loss: 0.5407247952638462
loss: 0.5345816935019381
loss: 0.5342413630261516
loss: 0.575
loss: 0.5481846911978995
loss: 0.5373501186007384
loss: 0.5382267449680069
loss: 0.5418353896887125
loss: 0.5379365293765395
loss: 0.537762857150693
loss: 0.5374609380090585
loss: 0.5381396588255576
loss: 0.5374142367069898
loss: 0.5374393245023671
loss: 0.5374293368677182
loss: 0.5374298041727226
loss: 0.537484810551079
loss: 0.5374344679042256
loss: 0.5374960558594546
loss: 0.5373939978220027
loss: 0.5374351040071697
loss: 0.5374025741398163
Th

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [281]:
##################
#INSERT CODE HERE#
##################
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

#make predictions with RandomForest model
forest = RandomForestClassifier()
forest.fit(X_train,y_train) #train model
forest_train_pred = forest.predict(X_train) #prediction on training data
forest_test_pred = forest.predict(X_test) #prediction on test data

#make predictions with DecisionTree model
tree = DecisionTreeClassifier(criterion="entropy", max_depth=1)
tree.fit(X_train,y_train) #train model
tree_train_pred = tree.predict(X_train) #prediction on training data
tree_test_pred = tree.predict(X_test) #prediction on test data

#accuracy of linear model
print("The accuracy with the linear model on the training data is:", accuracy(lm_train_pred,y_train))
print("The accuracy with the linear model on the test data is:", accuracy(lm_test_pred,y_test))

#accuracy of RandomForest model
print("The accuracy with the RandomForest on the training data is:", accuracy(forest_train_pred,y_train))
print("The accuracy with the RandomForest on the test data is:", accuracy(forest_test_pred,y_test))

#accuracy of DecisionTree model
print("The accuracy with the DecisionTree on the training data is:", accuracy(tree_train_pred,y_train))
print("The accuracy with the DecisionTree on the test data is:", accuracy(tree_test_pred,y_test))

The accuracy with the linear model on the training data is: 0.7375
The accuracy with the linear model on the test data is: 0.9197860962566845
The accuracy with the RandomForest on the training data is: 0.9375
The accuracy with the RandomForest on the test data is: 0.7754010695187166
The accuracy with the DecisionTree on the training data is: 0.725
The accuracy with the DecisionTree on the test data is: 0.6149732620320856
